<a href="https://colab.research.google.com/github/Aaron-Thai/g458-lab2/blob/main/Copy_of_Copy_of_02_geosearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [ ]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [ ]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "goes-in-the-quotes"
    consumer_secret = "goes-in-the-quotes"
    access_token = "goes-in-the-quotes"
    access_token_secret = "goes-in-the-quotes"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
                 -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
                 -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii

    stream_listener = StreamListener(time_limit=180, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, languages=['en'], encoding="utf-8",)

{'id': 1517355270405459968, 'username': 'MaryAnn26920610', 'created_at': 'Fri Apr 22 04:11:02 +0000 2022', 'lng': -121.469664, 'lat': 38.561722, 'text': '@vivianmuska Nice to see a smile'}
{'id': 1517355271307280387, 'username': 'Lindajjcleanerl', 'created_at': 'Fri Apr 22 04:11:02 +0000 2022', 'lng': -75.66877, 'lat': 41.4194095, 'text': 'I never been to Talladega but damn it sure sounds like fun!'}
{'id': 1517355273589071874, 'username': 'HerosLife', 'created_at': 'Fri Apr 22 04:11:02 +0000 2022', 'lng': -77.49317049999999, 'lat': 37.529883, 'text': 'Pope Hat'}
{'id': 1517355273500901376, 'username': 'MarcStormTTWN', 'created_at': 'Fri Apr 22 04:11:02 +0000 2022', 'lng': -100.0768885, 'lat': 31.1688935, 'text': "@ainee @arnelpineda Arnel &amp; I have the same birthday September 5th. That's one side of the sign I made. LoL 😆 https://t.co/ROVGPnwXld"}
{'id': 1517355275363225608, 'username': 'bigturnpike', 'created_at': 'Fri Apr 22 04:11:03 +0000 2022', 'lng': -83.804475, 'lat': 27.6986

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
